In [1]:
import sys,os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as nplin
import scipy as sp
from scipy.linalg import pinvh as spinv
from scipy.sparse import csr_matrix,csc_matrix,random
from sklearn.preprocessing import OneHotEncoder as OneHotEncoder
from sklearn.linear_model import lasso_path
#import Bio

In [2]:
def try_1hot():
    np.random.seed(7)

    n_seq,n_positions,n_residues = 6,4,3
    seqs = np.random.randint(0,n_residues,size=(n_seq,n_positions))
    print(seqs)
    print(seqs.shape)
    
    enc = OneHotEncoder(n_values=n_residues)
    onehot = csr_matrix(enc.fit_transform(seqs)).toarray()
    print(onehot)
    print(onehot.shape)

In [3]:
n_positions = 4
n_residues = 3
n_size = n_positions*n_residues
w = (np.random.rand(n_size,n_size)-0.5)
print(w)
print(w.shape)

[[-0.27249241 -0.10336884  0.45845218 -0.45339036 -0.48188976  0.04540235
   0.12818188 -0.08988634 -0.040684   -0.13807264  0.22037657  0.31810946]
 [ 0.48633539 -0.43922183  0.0863929  -0.32209777  0.0136194  -0.15141666
  -0.44162322  0.39018694 -0.33648824 -0.02887425  0.26895606  0.08058607]
 [ 0.48448189  0.00729131  0.31859137 -0.02310099  0.31424161 -0.20081538
  -0.08512045 -0.19323007  0.16853618 -0.13857414  0.24917632  0.28885038]
 [-0.01222083  0.05543246  0.43934029  0.40913851 -0.1892375  -0.30198342
   0.21130497 -0.48782494 -0.0430187  -0.09720002  0.17408563  0.11547318]
 [ 0.08622327 -0.21888216  0.1930626  -0.37205978  0.27981309 -0.39610862
   0.10639742  0.05058526  0.42714096  0.37392675  0.38101073 -0.47230323]
 [-0.49059685 -0.0072079   0.13547244 -0.01955924 -0.38354729 -0.28610529
  -0.34039347  0.13513434  0.08271338 -0.4155731  -0.26055415  0.27559094]
 [ 0.30818042 -0.38944064  0.02372086  0.22644474 -0.30866298 -0.20463538
  -0.33843867  0.38769409  0.391

In [4]:
b = np.random.rand(n_size)-0.5
print(b)
print(b.shape)

[ 0.18284688  0.32149339  0.03550157 -0.2471313  -0.07535068  0.04348305
  0.29971738  0.39592338 -0.04713683  0.25328309  0.48662529 -0.36357503]
(12,)


In [5]:
n_seq,n_positions,n_residues = 6,4,3
seqs = np.random.randint(0,n_residues,size=(n_seq,n_positions))
print(seqs)
print(seqs.shape)

enc = OneHotEncoder(n_values=n_residues)
onehot = csr_matrix(enc.fit_transform(seqs))
print(onehot)
print(onehot.shape)

[[1 2 1 2]
 [0 0 0 2]
 [2 1 2 2]
 [2 2 2 2]
 [1 1 2 1]
 [0 2 2 1]]
(6, 4)
  (0, 1)	1.0
  (0, 5)	1.0
  (0, 7)	1.0
  (0, 11)	1.0
  (1, 0)	1.0
  (1, 3)	1.0
  (1, 6)	1.0
  (1, 11)	1.0
  (2, 2)	1.0
  (2, 4)	1.0
  (2, 8)	1.0
  (2, 11)	1.0
  (3, 2)	1.0
  (3, 5)	1.0
  (3, 8)	1.0
  (3, 11)	1.0
  (4, 1)	1.0
  (4, 4)	1.0
  (4, 8)	1.0
  (4, 10)	1.0
  (5, 0)	1.0
  (5, 5)	1.0
  (5, 8)	1.0
  (5, 10)	1.0
(6, 12)


In [6]:
n_res = n_residues
index = 0

# convert matrix to array:
target = onehot.A[:,index*n_res:(index+1)*n_res]
print(target)

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [7]:
#remove column index in the sequence
def split_seqs(seqs,index): 
    return np.copy(np.hstack([seqs[:,:index],seqs[:,index+1:]]))

split_seqs0 = split_seqs(seqs,index)
print(split_seqs0)

[[2 1 2]
 [0 0 2]
 [1 2 2]
 [2 2 2]
 [1 2 1]
 [2 2 1]]


In [8]:
enc = OneHotEncoder(n_values=n_residues)
onehot = csr_matrix(enc.fit_transform(split_seqs0))
print(onehot)
print(onehot.shape)

  (0, 2)	1.0
  (0, 4)	1.0
  (0, 8)	1.0
  (1, 0)	1.0
  (1, 3)	1.0
  (1, 8)	1.0
  (2, 1)	1.0
  (2, 5)	1.0
  (2, 8)	1.0
  (3, 2)	1.0
  (3, 5)	1.0
  (3, 8)	1.0
  (4, 1)	1.0
  (4, 5)	1.0
  (4, 7)	1.0
  (5, 2)	1.0
  (5, 5)	1.0
  (5, 7)	1.0
(6, 9)


In [9]:
b_init = np.mean(target,axis=0)
print(b_init)

b_init_zero = b_init > 0.99
print(b_init_zero)

# change the value of positions that don't satisfy the above condition  
b_init[~b_init_zero] = 0.5/float(n_seq)
print(b_init)

[0.33333333 0.33333333 0.33333333]
[False False False]
[0.08333333 0.08333333 0.08333333]


In [10]:
b_init = np.log(b_init/np.sum(b_init))
print(b_init)

b[index*n_res:(index+1)*n_res] = b_init
print(b)

[-1.09861229 -1.09861229 -1.09861229]
[-1.09861229 -1.09861229 -1.09861229 -0.2471313  -0.07535068  0.04348305
  0.29971738  0.39592338 -0.04713683  0.25328309  0.48662529 -0.36357503]


In [11]:
def corr(onehot):
    #corr_out = onehot.transpose().dot(onehot).toarray()/np.float(onehot.shape[0])
    corr_out = np.cov(onehot.A,rowvar=False)
    expect = np.array(onehot.mean(axis=0)) #expectation value
    return expect,corr_out# - np.outer(expect,expect) #connected correlation function

expect_onehot,corr_onehot = corr(onehot)
print('expect_onehot:')
print(expect_onehot)

print('corr_onehot:')
print(corr_onehot)

expect_onehot:
[[0.16666667 0.33333333 0.5        0.16666667 0.16666667 0.66666667
  0.         0.33333333 0.66666667]]
corr_onehot:
[[ 0.16666667 -0.06666667 -0.1         0.16666667 -0.03333333 -0.13333333
   0.         -0.06666667  0.06666667]
 [-0.06666667  0.26666667 -0.2        -0.06666667 -0.06666667  0.13333333
   0.          0.06666667 -0.06666667]
 [-0.1        -0.2         0.3        -0.1         0.1         0.
   0.          0.          0.        ]
 [ 0.16666667 -0.06666667 -0.1         0.16666667 -0.03333333 -0.13333333
   0.         -0.06666667  0.06666667]
 [-0.03333333 -0.06666667  0.1        -0.03333333  0.16666667 -0.13333333
   0.         -0.06666667  0.06666667]
 [-0.13333333  0.13333333  0.         -0.13333333 -0.13333333  0.26666667
   0.          0.13333333 -0.13333333]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [-0.06666667  0.06666667  0.         -0.06666667 -0.06666667  0.13333333
   0.          0.2

In [12]:
corr_inv = spinv(corr_onehot)
print('corr_inv:')
print(corr_inv)

corr_inv:
[[ 0.79861111 -0.60763889 -0.19097222  0.79861111 -0.71180556 -0.08680556
   0.          0.20833333 -0.20833333]
 [-0.60763889  1.65798611 -1.05034722 -0.60763889  1.08506944 -0.47743056
   0.         -0.10416667  0.10416667]
 [-0.19097222 -1.05034722  1.24131944 -0.19097222 -0.37326389  0.56423611
   0.         -0.10416667  0.10416667]
 [ 0.79861111 -0.60763889 -0.19097222  0.79861111 -0.71180556 -0.08680556
   0.          0.20833333 -0.20833333]
 [-0.71180556  1.08506944 -0.37326389 -0.71180556  2.69965278 -1.98784722
   0.          0.52083333 -0.52083333]
 [-0.08680556 -0.47743056  0.56423611 -0.08680556 -1.98784722  2.07465278
   0.         -0.72916667  0.72916667]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.20833333 -0.10416667 -0.10416667  0.20833333  0.52083333 -0.72916667
   0.          1.25       -1.25      ]
 [-0.20833333  0.10416667  0.10416667 -0.20833333 -0.52083333  0.72916667
   0.         -1.25 

In [13]:
n_iter = 10
disc = np.zeros((n_iter))
discr_old = 1e12
print('discr_old:')
print(discr_old)

discr_old:
1e+12


In [14]:
h_seq = np.array(onehot.dot(np.vstack([w[:index*n_res,index*n_res:(index+1)*n_res],\
        w[(index+1)*n_res:,index*n_res:(index+1)*n_res]]))).reshape(n_seq,n_res)
print('h_seq:')
print(h_seq)

h_seq:
[[-0.66141831  0.49123356  0.31594081]
 [ 0.40847294 -0.23645695  0.4092915 ]
 [-0.1157497  -0.50139722  0.07574427]
 [-0.69256982 -0.28972296  0.0181541 ]
 [ 0.10261803 -1.08332487  0.62683667]
 [-0.47420209 -0.87165061  0.56924651]]


In [15]:
w.shape

(12, 12)

In [16]:
w1 = w[:index*n_res,index*n_res:(index+1)*n_res]
print('w1:')
print(w1)

w2 = w[(index+1)*n_res:,index*n_res:(index+1)*n_res]
print('w2:')
print(w2)
print(w2.shape)

w1:
[]
w2:
[[-0.01222083  0.05543246  0.43934029]
 [ 0.08622327 -0.21888216  0.1930626 ]
 [-0.49059685 -0.0072079   0.13547244]
 [ 0.30818042 -0.38944064  0.02372086]
 [-0.28333482  0.40089024  0.23423802]
 [-0.31448632 -0.38006628 -0.06354869]
 [-0.17909677  0.32846516  0.03802865]
 [ 0.33088108 -0.48437643  0.49732276]
 [ 0.11251335  0.09755122 -0.05376965]]
(9, 3)


In [17]:
w12 = np.vstack([w1,w2])
print('w12:')
print(w12)
print(w12.shape)

w12:
[[-0.01222083  0.05543246  0.43934029]
 [ 0.08622327 -0.21888216  0.1930626 ]
 [-0.49059685 -0.0072079   0.13547244]
 [ 0.30818042 -0.38944064  0.02372086]
 [-0.28333482  0.40089024  0.23423802]
 [-0.31448632 -0.38006628 -0.06354869]
 [-0.17909677  0.32846516  0.03802865]
 [ 0.33088108 -0.48437643  0.49732276]
 [ 0.11251335  0.09755122 -0.05376965]]
(9, 3)


In [18]:
onehotdot = onehot.dot(w12)
print('onehotdot:')
print(onehotdot)
print(onehotdot.shape)

onehotdot:
[[-0.66141831  0.49123356  0.31594081]
 [ 0.40847294 -0.23645695  0.4092915 ]
 [-0.1157497  -0.50139722  0.07574427]
 [-0.69256982 -0.28972296  0.0181541 ]
 [ 0.10261803 -1.08332487  0.62683667]
 [-0.47420209 -0.87165061  0.56924651]]
(6, 3)


In [19]:
print('onehot.A:')
print(onehot.A)

c = np.dot(onehot.A,w12)
print(c)

onehot.A:
[[0. 0. 1. 0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 0. 1. 0. 1. 0.]]
[[-0.66141831  0.49123356  0.31594081]
 [ 0.40847294 -0.23645695  0.4092915 ]
 [-0.1157497  -0.50139722  0.07574427]
 [-0.69256982 -0.28972296  0.0181541 ]
 [ 0.10261803 -1.08332487  0.62683667]
 [-0.47420209 -0.87165061  0.56924651]]


In [20]:
b[index*n_res:(index+1)*n_res] -= np.mean(b[index*n_res:(index+1)*n_res])
print('b:')
print(b)

b:
[ 0.          0.          0.         -0.2471313  -0.07535068  0.04348305
  0.29971738  0.39592338 -0.04713683  0.25328309  0.48662529 -0.36357503]


In [21]:
expect_h = np.mean(h_seq,axis=0)
print(expect_h)

[-0.23880816 -0.41521984  0.33586898]


In [22]:
res_index = 0
res_index1 = 1
tgt = target[:,res_index]-target[:,res_index1]

print(tgt)

[-1.  1.  0.  0. -1.  1.]


In [23]:
#remove sequences with tgt = 0:
which_BC = tgt != 0.0
print(which_BC)

tgt = tgt[which_BC]
print(tgt)

n_BC = float(tgt.shape[0])
print(n_BC)

print(h_seq)
print(h_seq[which_BC])

h_BC = 0.5*(h_seq[which_BC,res_index] - h_seq[which_BC,res_index1])

[ True  True False False  True  True]
[-1.  1. -1.  1.]
4.0
[[-0.66141831  0.49123356  0.31594081]
 [ 0.40847294 -0.23645695  0.4092915 ]
 [-0.1157497  -0.50139722  0.07574427]
 [-0.69256982 -0.28972296  0.0181541 ]
 [ 0.10261803 -1.08332487  0.62683667]
 [-0.47420209 -0.87165061  0.56924651]]
[[-0.66141831  0.49123356  0.31594081]
 [ 0.40847294 -0.23645695  0.4092915 ]
 [ 0.10261803 -1.08332487  0.62683667]
 [-0.47420209 -0.87165061  0.56924651]]


In [29]:
expect_h_zero = h_BC != 0.0
print(expect_h_zero)

ratio = 2*np.copy(tgt)
print(ratio)

ratio[expect_h_zero] *= h_BC[expect_h_zero]/np.tanh(h_BC[expect_h_zero])
print(ratio)

ratio_BsumC = ratio/n_BC #so each sequence which is either B or C counts once
print(ratio_BsumC)

[ True  True  True  True]
[-2.  2. -2.  2.]
[-2.21668119  2.06884658 -2.22909304  2.0262585 ]
[-0.5541703   0.51721164 -0.55727326  0.50656463]


In [33]:
w_temp = onehot[which_BC,:].transpose().dot(ratio_BsumC-np.mean(ratio_BsumC))
print(w_temp)
print(len(w_temp))

[ 0.53912847 -0.53535644 -0.00377203  0.53912847 -0.53225348 -0.00687499
  0.         -0.00687499  0.00687499]
9


In [35]:
n_pos = 4

w_temp = np.zeros((n_res*(n_pos-1)))
print(w_temp.shape)

(9,)
